# MCT4001 Scientific Computing in Python Session 4
![mct-banner](https://raw.githubusercontent.com/wiki/MCT-master/Guides/assets/img/mct-banner.jpg)

In [ ]:
# enabling auto completion
%config IPCompleter.greedy=True

#importing packages
import os

## 1. Python and Pure Data Integration

In order to run the following code cells in on your computer you must update the path to Pure Data executable.

In [ ]:
# this must be updated to the actual path on your computer
pd_executable = '/Applications/Pd-0.51-1.app/Contents/Resources/bin/pd' #for OSX Pd 0.51-1
#pd_executable = 'C:\Program Files\pd\bin\pd.exe' #for Windows
#pd_executable = '/usr/local/bin/pd ' #for Linux

#running PD - you have to close it manually after running this cell
os.system(pd_executable)

In [ ]:
#running 5secSineToDac.pd
pd_patch = 'files/puredata/5secSineToDac.pd'
command = pd_executable + ' -open ' + pd_patch #for convenience, combining the command in a single string
print(command) #checking the correctness of the command string before using it in os.system()
os.system(command)

In [ ]:
#running 5secSineToDac.pd without GUI
pd_patch = 'files/puredata/5secSineToDac.pd'
command = pd_executable + ' -open ' + pd_patch + ' -nogui'
print(command)
os.system(command)

In [ ]:
#running 5secSineToDac.pd without GUI passing frequency value
pd_patch = 'files/puredata/5secSineToDacVar.pd'
command = pd_executable + ' -open ' + pd_patch + ' -send "; freqV 1000"' + ' -nogui'
print(command)
os.system(command)

In [ ]:
#running 5secSineToWav.pd without GUI
pd_patch = 'files/puredata/5secSineToWav.pd'
command = pd_executable + ' -open ' + pd_patch + ' -nogui'
print(command)
os.system(command)

In [ ]:
#running 5secSineToWav.pd without GUI offline as batch process
pd_patch = 'files/puredata/5secSineToWav.pd'
command = pd_executable + ' -open ' + pd_patch + ' -nogui' + ' -batch'
print(command)
os.system(command)

In [ ]:
#running SineToWavVar.pd without GUI offline as batch process passing values for frequency, duration and filename
pd_patch = 'files/puredata/SineToWavVar.pd'
send =  ' -send "; freqV 5000; nameV myfile.wav; durV 10000"'
command = pd_executable + ' -open ' + pd_patch + send + ' -nogui' + ' -batch'
print(command)
os.system(command)

In [ ]:
#creating a loop that generate a collection of wave files with different frequencies
#files be further loaded and processed in Python either within the loop or (better) after the loop
pd_patch = 'files/puredata/SineToWavVar.pd'

for i in range(100,500,10):
    send =  ' -send "; freqV ' + str(i) + '; nameV out' + str(i) + '.wav; durV 10000"'
    command = pd_executable + ' -open ' + pd_patch + send + ' -nogui' + ' -batch'
    #print(command)
    os.system(command)

## 2. Python OSC

#### Python OSC client example

In [ ]:
#run this cell after openning files/puredata/OSCserver.pd
import time
from pythonosc import udp_client

#creating OSC client that will send messages to 127.0.0.1 or localhost - i.e. same machinne - on port 8002
client = udp_client.SimpleUDPClient('127.0.0.1', 8002)

for x in range(5):
    client.send_message('/my_address', x) #sending the index of the loop with specified OSC address
    time.sleep(1) #doig nothing for 1 secod

#### Python OSC server example

In [ ]:
#run this cell and then open files/puredata/OSCclient.pd, click on connect and send messages
from pythonosc import dispatcher
from pythonosc import osc_server

#creating a function that will handle the  OSC received data, just printing inn this case
def handler(address, args):
    print('address is',address)
    print('values are')
    print(args)
    print('')
    

#attaching the message handling function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("*", handler) #with the wildcard * the handler function will handle any message

#starting initializing and  starting  the OSC server
server = osc_server.ThreadingOSCUDPServer(('127.0.0.1', 8001), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
#closing the OSC server
#if we do not do this, next time we try to create a server listenign on the same port we will get an error
server.server_close()

#### Python OSC server-client example

The following python cell receives data from Pure Data via OSC, process the ddata, and send it back to Pure data via OSC. On both sides, we are using a OSC client and server. 

In [ ]:
#run this cell and then open files/puredata/OSCclient-server.pd, click on connect, send messages, and observe messages sent back

from pythonosc import dispatcher
from pythonosc import udp_client
from pythonosc import osc_server
from IPython import display
import math


#creating OSC client that will send messages to 127.0.0.1 or localhost - i.e. same machinne - on port 8002
client = udp_client.SimpleUDPClient('127.0.0.1', 8002)


#creating a functions that will handle the OSC received data
def slider1_handler(address, args):
    #display.clear_output(wait=True) #this function simply clears the standard, then we print the most recent received OSC message
    #print('S1', args)
    out = abs(args-1)
    client.send_message('/invslider1', out)
    
def slider2_handler(address, args):
    #display.clear_output(wait=True) #this function simply clears the standard, then we print the most recent received OSC message
    #print('S1', args)
    out = math.log(1+100*args)/math.log(100)
    #print(out)
    client.send_message('/logslider2', out)

#attaching the message handling function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/slider1*", slider1_handler) #attaching handler to dispatcher
dispatcher.map("/slider2*", slider2_handler) #attaching handler to dispatcher

#starting initializing and  starting  the OSC server
server = osc_server.ThreadingOSCUDPServer(('127.0.0.1', 8001), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()
    

In [ ]:
#closing the OSC server
#if we do not do this, next time we try to create a server listenign on the same port we will get an error
server.server_close()